In [278]:
from paddleocr import PaddleOCR
import pandas as pd
import numpy as np
import unicodedata
import json
import re
import cv2
import os
import re

pd.set_option('future.no_silent_downcasting', True)

print("Librerias Cargadas")

Librerias Cargadas


In [279]:
# ocr = PaddleOCR(
#     use_doc_orientation_classify=False, 
#     use_doc_unwarping=False, 
#     use_textline_orientation=False) # text detection + text recognition

# text image preprocessing + text detection + textline orientation classification + text recognition
ocr = PaddleOCR(use_doc_orientation_classify=False, use_doc_unwarping=False, lang="es")

# ocr = PaddleOCR(use_doc_orientation_classify=False, use_doc_unwarping=False) # text detection + textline orientation classification + text recognition
# ocr = PaddleOCR(
#     text_detection_model_name="PP-OCRv5_mobile_det",
#     text_recognition_model_name="PP-OCRv5_mobile_rec",
#     use_doc_orientation_classify=False,
#     use_doc_unwarping=False,
#     use_textline_orientation=False) # Switch to PP-OCRv5_mobile models

Creating model: ('PP-LCNet_x1_0_textline_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/root/.paddlex/official_models/PP-LCNet_x1_0_textline_ori`.
Creating model: ('PP-OCRv5_server_det', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/root/.paddlex/official_models/PP-OCRv5_server_det`.
Creating model: ('latin_PP-OCRv5_mobile_rec', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/root/.paddlex/official_models/latin_PP-OCRv5_mobile_rec`.


In [280]:
image_dir = "./imagenes"  # ruta a tu carpeta con imágenes
image_extensions = [".jpg", ".jpeg", ".png", ".bmp"]

# Filtramos solo imágenes
image_files = sorted(
    [os.path.join(image_dir, f) 
     for f in os.listdir(image_dir) 
     if os.path.splitext(f)[1].lower() in image_extensions],
    key=lambda x: int(os.path.splitext(os.path.basename(x))[0])
)

print(str(len(image_files))+" Imagenes Cargadas")

11 Imagenes Cargadas


In [281]:
def clean_text_simple(text):
    if not isinstance(text, str):
        return text
     # lista por defecto a eliminar (puedes ampliarla)
    default_remove = ["continua...", "continua", "ejemplo", "borrar", "...",
    "DEDUCIR",
    "EL",
    "COSTO",
    "DE",
    "REACONDICIONAMIENTO",
    "Linea",
    "Nueva",
    "Unidades", "Usadas","-","Actualizacion",
    "Unidades",
    "Nuevas",
    ".",
    "Precios",
    ":",
    "Lista",
    "-",
    "N D."
    ]
    removes = default_remove

    s = text.strip()

    # normalizar y eliminar diacríticos (á -> a, ñ -> n, etc.)
    s = unicodedata.normalize("NFKD", s)
    s = s.encode("ascii", "ignore").decode("ascii")

    # eliminar apostrofes rectos y curvos y guion bajo
    s = re.sub(r"[\'\u2019_]", "", s)

    # construir patrón para eliminar palabras/frases como palabras completas
    # normalizamos las frases a ascii (ya lo hicimos arriba) y escapamos
    # usamos boundaries \b para evitar quitar partes dentro de otras palabras
    escaped = [re.escape(r) for r in removes if r]
    if escaped:
        pattern = r"\b(?:" + "|".join(escaped) + r")\b"
        s = re.sub(pattern, " ", s, flags=re.IGNORECASE)

    # remover múltiples espacios y strip final
    s = re.sub(r"\s+", " ", s).strip()
    return s

In [ ]:
def ocr_to_multidimensional_sections(result, line_gap=6.5, cortes=None):
    texts = result["rec_texts"]
    boxes = result["rec_boxes"]

    # Extraer coordenadas y asociar texto
    data = []
    for text, box in zip(texts, boxes):
        x_min, y_min = box[0], box[1]  # esquina superior izq
        text = clean_text_simple(text)
        data.append({"text": text, "x": x_min, "y": y_min})

    # Ordenar primero por Y (vertical) y luego por X (horizontal)
    data = sorted(data, key=lambda d: (d["y"], d["x"]))

    # Agrupamos por líneas (Y)
    ordered_lines = []
    current_line = []
    last_y = None

    for item in data:
        if last_y is None or abs(item["y"] - last_y) <= line_gap:
            current_line.append(item)
        else:
            ordered_lines.append(current_line)
            current_line = [item]
        last_y = item["y"]

    if current_line:
        ordered_lines.append(current_line)

    # Si no se pasan cortes, devolvemos normal
    if not cortes:
        max_len = max(len(line) for line in ordered_lines)
        ordered_filled = [
            [i["text"] for i in line] + [""] * (max_len - len(line))
            for line in ordered_lines
        ]
        return pd.DataFrame(ordered_filled)

    # --- Usar cortes en X para dividir en secciones ---
    cortes = sorted(cortes)  # [178, 286, 373] por ejemplo
    n_sections = len(cortes) + 1

    all_rows = []
    for line in ordered_lines:
        row = [""] * n_sections
        for item in line:
            x = item["x"]
            text = item["text"]

            # Buscar en qué sección cae
            section_idx = 0
            for c in cortes:
                if x > c:
                    section_idx += 1
                else:
                    break
            row[section_idx] += (" " + text if row[section_idx] else text)

        all_rows.append(row)

    df = pd.DataFrame(all_rows)
    return df

In [283]:
def crear_imagenes_con_lineas(promedios):
    output_dir = "./imagenes_con_lineas"
    os.makedirs(output_dir, exist_ok=True)
    
    for img_path in image_files:
        img = cv2.imread(img_path)
        if img is None:
            print(f"❌ No se pudo leer: {img_path}")
            continue
    
        # Obtener dimensiones
        height, width, _ = img.shape
    
        # Dibujar cada línea
        for x in promedios:
            color = (0, 255, 0)  # Verde
            thickness = 2
            cv2.line(img, (int(x), 0), (int(x), height), color, thickness)
    
        # Guardar nueva imagen
        output_path = os.path.join(output_dir, os.path.basename(img_path))
        cv2.imwrite(output_path, img)
        print(f"✅ Guardada: {output_path}")


In [284]:
all_texts = {}

promedios = [120,900,1250]

resp = input("¿Deseas continuar? (s/n): ").strip().lower()
if resp in ("s", "si", "y", "yes"):
    print("Usuario dijo SÍ")
    crear_imagenes_con_lineas(promedios)
else:
    print("Usuario dijo NO")

for img_path in image_files[:10]:
    print(img_path)
    res = ocr.predict(img_path)

    df_to_add = ocr_to_multidimensional_sections(res[0], line_gap=6.5, cortes=promedios)
    #print(df_to_add)
    all_texts[os.path.basename(img_path)] = df_to_add
#print(all_texts)

Usuario dijo NO
./imagenes/20.jpg
./imagenes/21.jpg
./imagenes/22.jpg
./imagenes/23.jpg
./imagenes/24.jpg
./imagenes/25.jpg
./imagenes/26.jpg
./imagenes/27.jpg
./imagenes/28.jpg
./imagenes/29.jpg


# Guardar todo en un solo Excel

In [285]:
excel_path = "ocr_resultado_completo.xlsx"

# Concatenar todos los DataFrames en uno solo
final_df = pd.concat(all_texts.values(), ignore_index=True)

# Eliminar filas donde TODAS las columnas estén vacías o con espacios
final_df = final_df.replace(r'^\s*$', np.nan, regex=True)  # convierte espacios en NaN
final_df = final_df.dropna(how='all')  # elimina filas donde todo es NaN

resp_save_original = input("¿Deseas continuar? (s/n): ").strip().lower()
if resp_save_original in ("s", "si", "y", "yes"):
    print("Usuario dijo SÍ")
    final_df.to_excel("ocr_origen.xlsx", index=False, header=False)
else:
    print("Usuario dijo NO")


Usuario dijo NO


In [286]:
def leer_data_in_json(path):
    with open(path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

In [287]:
def cargar_datos_referencia(json_path):
    """Carga y procesa el JSON de marcas y modelos"""
    json_data = leer_data_in_json(json_path)["rows"]
    
    marcas_validas = set()
    modelos_por_marca = {}
    
    for row in json_data:
        marca = row["marca"].upper()
        modelo = row["modelo"].upper()
        
        marcas_validas.add(marca)
        if marca not in modelos_por_marca:
            modelos_por_marca[marca] = set()
        modelos_por_marca[marca].add(modelo)
    
    return marcas_validas, modelos_por_marca

In [288]:
def buscar_en_fila(row, columnas_indices, valores_buscar):
    """
    Busca valores en múltiples columnas de una fila
    
    Args:
        row: fila del DataFrame
        columnas_indices: lista de índices de columnas a buscar [0,1,2,3]
        valores_buscar: set de valores a buscar
    
    Returns:
        valor encontrado o None
    """
    for idx in columnas_indices:
        if idx < len(row):
            valor_raw = row.iloc[idx]
            if pd.notna(valor_raw):
                valor = str(valor_raw).strip().upper()
                if valor and valor in valores_buscar:
                    return valor
    return None

In [289]:
def detectar_marcas_modelos(df, marcas_validas, modelos_por_marca, columnas=[0, 1, 2, 3]):
    """
    Detecta y propaga marcas y modelos en el DataFrame
    
    Args:
        df: DataFrame a procesar
        marcas_validas: set de marcas válidas
        modelos_por_marca: dict {marca: set(modelos)}
        columnas: lista de índices de columnas donde buscar
    
    Returns:
        tuple (lista_marcas, lista_modelos)
    """
    marcas = []
    modelos = []
    marca_actual = None
    modelo_actual = None
    
    for idx, row in df.iterrows():
        # Buscar marca en cualquier columna
        marca_encontrada = buscar_en_fila(row, columnas, marcas_validas)
        if marca_encontrada:
            marca_actual = marca_encontrada
            # ✅ Solo resetea si la marca REALMENTE cambió
            if marca_actual != (marcas[-1] if marcas else None):
                modelo_actual = None
            print(f"Fila {idx}: Nueva marca → {marca_actual}")

        # Buscar modelo
        if marca_actual and marca_actual in modelos_por_marca:
            modelo_encontrado = buscar_en_fila(row, columnas, modelos_por_marca[marca_actual])
            if modelo_encontrado:
                modelo_actual = modelo_encontrado
                print(f"Fila {idx}: Nuevo modelo → {modelo_actual}")
            # ✅ Si no encuentra modelo, mantiene modelo_actual (propagación)
        
        marcas.append(marca_actual)
        modelos.append(modelo_actual)
    
    return marcas, modelos

In [290]:
def separar_anio_y_resto_mejorado(texto):
    """Separa año del resto del texto, detectando patrones como 2024, 2023Q2, etc."""
    if pd.isna(texto) or texto == "":
        return pd.Series([np.nan, ""])
    
    texto_str = str(texto).strip()
    
    import re
    
    # Patrón 1: Año seguido de Q y número (2023Q2, 2024Q1, etc.)
    match = re.match(r'^(\d{4})Q\d+', texto_str, re.IGNORECASE)
    if match:
        anio = match.group(1)
        print(f"  Detectado año con trimestre: {texto_str} → año: {anio}")
        return pd.Series([anio, ""])
    
    # Patrón 2: Año seguido de espacio y texto (2024 5p Dynamic...)
    match = re.match(r'^(\d{4})\s+(.+)', texto_str)
    if match:
        return pd.Series([match.group(1), match.group(2)])
    
    # Patrón 3: Solo año (2024, 2023, etc.)
    match = re.match(r'^(\d{4})$', texto_str)
    if match:
        return pd.Series([match.group(1), ""])
    
    return pd.Series([np.nan, texto_str])

In [291]:
def procesar_excel(input_df, json_path, output_path):
    """Función principal que procesa el Excel completo"""
    
    # 1. Cargar datos de referencia
    print("=== Cargando datos de referencia ===")
    marcas_validas, modelos_por_marca = cargar_datos_referencia(json_path)
    print(f"Marcas encontradas: {len(marcas_validas)}")
    print(f"Total de modelos: {sum(len(v) for v in modelos_por_marca.values())}\n")
    
    # 2. Copiar DataFrame
    df = input_df.copy()
    
    # 3. Detectar marcas y modelos en columnas A, B, C, D
    print("=== Detectando marcas y modelos ===")
    marcas, modelos = detectar_marcas_modelos(
        df, 
        marcas_validas, 
        modelos_por_marca,
        columnas=[0, 1, 2, 3]  # A, B, C, D
    )
    
    df["marca"] = marcas
    df["modelo"] = modelos
    
    # 4. Detectar año y texto restante
    print("\n=== Procesando años ===")
    anio_resto = df.iloc[:, 1].apply(separar_anio_y_resto_mejorado)
    df["año"] = anio_resto[0]
    df["texto"] = anio_resto[1]
    
    # 5. Propagar año hacia abajo
    df["año"] = df["año"].ffill()
    
    # 6. Detectar versiones
    print("\n=== Detectando versiones ===")
    versiones = []
    for idx, texto in enumerate(df["texto"]):
        modelo_actual = df["modelo"].iloc[idx]
        
        if modelo_actual and isinstance(texto, str) and texto.strip():
            texto_upper = texto.strip().upper()
            # Si el texto NO es el modelo, es una versión
            if texto_upper != modelo_actual:
                versiones.append(texto.strip())
            else:
                versiones.append(np.nan)
        else:
            versiones.append(np.nan)
    
    df["version"] = versiones
    
     # 7. Añadir columnas de valores originales (columnas 2 y 3 = C y D)
    df["valor_c"] = df.iloc[:, 2] if len(df.columns) > 2 else np.nan
    df["valor_d"] = df.iloc[:, 3] if len(df.columns) > 3 else np.nan
    
    # 8. Reordenar columnas
    columnas_base = ["marca", "modelo", "año", "version", "valor_c", "valor_d"]
    columnas_extras = [col for col in df.columns 
                       if col not in columnas_base + [0, 1, 2, 3, "texto"]]
    
    excel_df = df[columnas_base + columnas_extras]
    
    # 8. Guardar
    excel_df.to_excel(output_path, index=False, header=False)
    
    print(f"\n¡Listo! Archivo guardado en: {output_path}")
    print(f"\nEstadísticas:")
    print(f"- Total filas procesadas: {len(excel_df)}")
    print(f"- Marcas únicas detectadas: {excel_df['marca'].nunique()}")
    print(f"- Modelos únicos detectados: {excel_df['modelo'].nunique()}")
    
    return excel_df


In [292]:
# =====================
# EJECUCIÓN PRINCIPAL
# =====================

# Usar la función original separar_anio_y_resto si ya la tienes definida
# Si no, se usa la versión mejorada de arriba

resultado = procesar_excel(
    input_df=final_df,
    json_path="data.json",
    output_path=excel_path
)

# Mostrar primeras filas
print("\n=== Primeras 20 filas del resultado ===")
print(resultado.head(20))

=== Cargando datos de referencia ===
Marcas encontradas: 111
Total de modelos: 1525

=== Detectando marcas y modelos ===
Fila 0: Nueva marca → AUDI
Fila 1: Nuevo modelo → A5
Fila 85: Nueva marca → AUDI
Fila 209: Nueva marca → AUDI
Fila 228: Nuevo modelo → A6
Fila 315: Nueva marca → AUDI
Fila 349: Nuevo modelo → A7
Fila 406: Nueva marca → AUDI
Fila 445: Nuevo modelo → A8
Fila 490: Nueva marca → AUDI
Fila 609: Nueva marca → AUDI
Fila 725: Nueva marca → AUDI
Fila 798: Nueva marca → AUDI
Fila 932: Nueva marca → AUDI

=== Procesando años ===
  Detectado año con trimestre: 2023Q2 → año: 2023
  Detectado año con trimestre: 2021Q2 → año: 2021
  Detectado año con trimestre: 2016Q5 → año: 2016
  Detectado año con trimestre: 2015Q5 → año: 2015
  Detectado año con trimestre: 2023Q7 → año: 2023
  Detectado año con trimestre: 2022Q7 → año: 2022

=== Detectando versiones ===

¡Listo! Archivo guardado en: ocr_resultado_completo.xlsx

Estadísticas:
- Total filas procesadas: 995
- Marcas únicas detectad